In [17]:
import numpy as np
import pandas as pd
import os
import re
from PIL import Image, ImageDraw

# Code for df prep

In [18]:
df = pd.read_csv('/Users/agnia.sergeyuk/PycharmProjects/Readability-Features/code/snips.csv')
df

,snip_id,task,snippet
0,1,A happy number is defined by the following Seq...,public class HappyNumbers {\n public static...
1,2,A Leyland number is of the form (x of degree y...,public class LeylandNumbers {\n public stat...
2,3,A Niven number is a positive integer that is d...,public class NivenNumbers {\n public static...
3,4,A number is a divisor of another number if it ...,public class Divisors {\n public static voi...
4,5,A pangram is a sentence that uses every letter...,public class PangramValidator{\n public sta...
5,6,A pernicious number is a positive number where...,public class PerniciousNumbers {\n public s...
6,7,A quine is a non-empty computer program which ...,public class Quine {\n public static void m...
7,8,A Smith number is a composite number whose dig...,public class SmithNumber {\n public static ...
8,9,A vampire number is a non-negative number that...,import java.util.*;\n\npublic class VampireNum...
9,10,An abundant number is a number for which the s...,public class AbundantNumbers {\n\n public s...


In [19]:
# #to find snips longer than 50 lines of code
# def count_lines(code_snippet):
#     lines = code_snippet.split('\n')
#     return len(lines)
# 
# df["line_count"] = df["snippet"].apply(count_lines)
# 
# # Get snip_id values where the code_snippet has more than 50 lines
# result = df[df["line_count"] > 50]["snip_id"].tolist()
# 
# print(result)

In [20]:
# #to extract possible classes for coloring

# def extract_classes_from_snippet(snippet):
#     classes = []
#     try:
#         tree = javalang.parse.parse(snippet)
#         for path, node in tree.filter(javalang.tree.ReferenceType):
#             classes.append(node.name)
#         return classes
#     except javalang.parser.JavaSyntaxError:
#         print("Syntax error while parsing snippet {row.name}")
#         return []

# def find_classes_in_df(df):
#     classes = []

#     for _, row in df.iterrows():
#         snippet = row['snippet']
#         classes.extend(extract_classes_from_snippet(snippet))

#     # Convert classes list to a set to remove duplicates
#     unique_classes = set(classes)
    
#     return unique_classes

# unique_classes = find_classes_in_df(df)
# print(unique_classes)

# Code for snip-img transformation

In [21]:
CLASSES_COLOR = (37, 74, 243)
IDENTIFIERS_COLOR = (109, 251, 251)
LITERALS_COLOR = (109, 251, 251)
OPERATORS_SEPARATORS_COLOR = (247, 249, 22)
KEYWORDS_COLOR = (247, 14, 12)
COMMENTS_COLOR = (58, 129, 29)
WHITESPACE_COLOR = (255, 255, 255)
STRINGS_COLOR = (115,244,64)

COMMENT_REGEX = r'^\s*(\/\/.*|\/\*.*?\*\/|\/\*\*[\s\S]*?\*\/)'
OPERATOR_SEPARATOR_REGEX = r'[-+*/%=<>!&|~^{}();.,]|[\(\)]'
IDENTIFIER_REGEX = r'[A-Za-z_]\w*'
LITERAL_REGEX = r'\d+(\.\d+)?'
STRING_REGEX = r'"[^"]*"'
CLASSES = r'\b[A-Z][A-Za-z]*[a-z][A-Za-z]*\b'

KEYWORDS = ["abstract", "assert", "boolean", "break", "byte",
            "case", "catch", "char", "class", "const", "continue",
            "default", "do", "double", "else", "enum", "extends", "false",
            "final", "finally", "float", "for", "if", "implements",
            "import", "instanceof", "int", "interface", "long", "native",
            "new", "null", "package", "private", "protected", "public", "return",
            "short", "static", "strictfp", "super", "switch", "synchronized",
            "this", "throw", "throws", "transient", "true", "try", "void", "volatile", 
            "while"]

CHARACTER_WIDTH = 11
CHARACTER_HEIGHT = 22
MARGIN = 10

In [22]:
def determine_color(word):
    if word.lower() in KEYWORDS:
        return KEYWORDS_COLOR
    elif re.match(OPERATOR_SEPARATOR_REGEX, word):
        return OPERATORS_SEPARATORS_COLOR
    elif re.match(IDENTIFIER_REGEX, word):
        if re.match(CLASSES, word):
            return CLASSES_COLOR
        else:
            return IDENTIFIERS_COLOR
    elif re.match(LITERAL_REGEX, word):
        return LITERALS_COLOR
    elif re.match(STRING_REGEX, word):
        return STRINGS_COLOR
    else:
        return WHITESPACE_COLOR

In [23]:
def generate_code_image(code):
    lines = code.splitlines()
    words = [re.findall(
        r'\".*?\"|\b[A-Za-z_][A-Za-z0-9_]*\b|\d+\.\d+|\d+|[+\-*/%=<>!&|~^{}();.,]|\s', line) for line in lines]
    image_width = max(len(line) for line in lines) * CHARACTER_WIDTH + 2 * MARGIN
    image_height = len(lines) * CHARACTER_HEIGHT + 2 * MARGIN
    image = Image.new("RGB", (image_width, image_height), "white")
    draw = ImageDraw.Draw(image)

    x, y = MARGIN, MARGIN
    javadoc_comment = False

    for line_number, line in enumerate(words):
        if re.match(r'^\s*\/\*\*', lines[line_number]):
            javadoc_comment = True

        if javadoc_comment or re.match(COMMENT_REGEX, lines[line_number], re.DOTALL):
            indentation_width = len(lines[line_number]) - len(lines[line_number].lstrip())
            width = len(lines[line_number]) * CHARACTER_WIDTH
            draw.rectangle(
                [(x + indentation_width * CHARACTER_WIDTH, y), (x + width, y + CHARACTER_HEIGHT)],
                fill=COMMENTS_COLOR, outline=None)

            if re.match(r'.*\*\/\s*$', lines[line_number]):
                javadoc_comment = False

        else:
            for word in line:
                color = determine_color(word)
                width = len(word) * CHARACTER_WIDTH
                draw.rectangle([(x, y), (x + width, y + CHARACTER_HEIGHT)], fill=color, outline=None)
                x += width

        x = MARGIN
        y += CHARACTER_HEIGHT

    try:
        return image
    except IOError:
        print("Error saving the image file.")

# Code for snip-matrix transformation

In [24]:
def ascii_matrix(code, rows=50, cols=350):
    matrix = [[-1]*cols for _ in range(rows)]
    
    for i, line in enumerate(code):
        if i == rows:
            break

        for j, char in enumerate(line):
            if j == cols:
                break

            matrix[i][j] = ord(char)
    
    return matrix

In [25]:
def process_snippet(snippet):
    lines = snippet.split('\n')
    return ascii_matrix(lines)

# Code for df-folders transformation

In [26]:
def save_data(df, folder, data_type):
    # Create the "OUR_Dataset" folder if it does not exist
    if not os.path.exists("OUR_Dataset"):
        os.makedirs("OUR_Dataset")

    # Determine the parent folder path based on the data_type
    if data_type == 'java':
        parent_folder_path = "OUR_Dataset"
    else:
        # Create a folder named "Processed Dataset" inside the "OUR_Dataset" folder if it does not exist
        processed_dataset_folder = os.path.join("OUR_Dataset", "Processed Dataset")
        if not os.path.exists(processed_dataset_folder):
            os.makedirs(processed_dataset_folder)
        parent_folder_path = processed_dataset_folder

    subfolder_path = os.path.join(parent_folder_path, folder)

    if not os.path.exists(subfolder_path):
        os.makedirs(subfolder_path)

    for index, row in df.iterrows():
        code_snippet = row['snippet']

        if data_type == 'image':
            data = generate_code_image(code_snippet)
            file_path = os.path.join(subfolder_path, f'{row["snip_id"]}.png')
            data.save(file_path)
        elif data_type == 'matrix':
            data = process_snippet(code_snippet)
            file_path = os.path.join(subfolder_path, f'{row["snip_id"]}.java.matrix')
            np.savetxt(file_path, data, fmt='%d')
        elif data_type == 'text':
            data = code_snippet
            file_path = os.path.join(subfolder_path, f'{row["snip_id"]}.txt')
            with open(file_path, 'w') as f:
                f.write(data)

        elif data_type == 'java':
            data = code_snippet
            file_path = os.path.join(subfolder_path, f'{row["snip_id"]}.java')
            with open(file_path, 'w') as f:
                f.write(data)
        else:
            raise ValueError(f"Invalid data_type: {data_type}. Choose from 'image', 'matrix', 'text', or 'java'.")



In [27]:
save_data(df, 'Image', 'image')  # To save as images
save_data(df, 'Structure', 'matrix')  # To save as .java.matrix files
save_data(df, 'Texture', 'text')  # To save as .txt files
save_data(df, 'Java', 'java')  # To save as .java files